In [42]:
import numpy as np
from scipy.stats import norm
import math

In [2]:
# Formule de Black-Scholes pour le prix d'un call
def black_scholes_call(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    call_price = S * N_d1 - K * np.exp(-r * T) * N_d2
    return call_price

# Volatilité implicite avec l'algorithme de Newton-Raphson
def implied_volatility_newton_raphson(option_price, S, K, r, T, initial_volatility, max_iter=100, tolerance=1e-6):
    sigma = initial_volatility
    for i in range(max_iter):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        N_d1 = norm.cdf(d1)
        vega = S * np.sqrt(T) * norm.pdf(d1)
        call_price = S * N_d1 - K * np.exp(-r * T) * norm.cdf(d2)
        diff = call_price - option_price
        if abs(diff) < tolerance:
            return sigma
        sigma = sigma - diff / vega
    return None

# Volatilité implicite avec l'algorithme de dichotomie
def implied_volatility_bisection(option_price, S, K, r, T, sigma_min, sigma_max, tolerance=1e-6):
    while (sigma_max - sigma_min) > tolerance:
        sigma_mid = (sigma_min + sigma_max) / 2
        call_price_mid = black_scholes_call(S, K, r, T, sigma_mid)
        if call_price_mid < option_price:
            sigma_min = sigma_mid
        else:
            sigma_max = sigma_mid
    return (sigma_min + sigma_max) / 2

In [5]:
S = 100  # Prix actuel du sous-jacent
K = 100  # Prix d'exercice de l'option
r = 0.05  # Taux d'intérêt sans risque
T = 1.0  # Temps jusqu'à l'expiration (en années)
option_price = 10.0  # Prix observé de l'option
initial_volatility = 0.2  # Valeur initiale de la volatilité
sigma_min = 0.01  # Valeur minimale possible de la volatilité
sigma_max = 1.0  # Valeur maximale possible de la volatilité

implied_vol_newton = implied_volatility_newton_raphson(option_price, S, K, r, T, initial_volatility)
implied_vol_bisection = implied_volatility_bisection(option_price, S, K, r, T, sigma_min, sigma_max)

print("Volatilité implicite (Newton-Raphson) : {:.4f}".format(implied_vol_newton))
print("Volatilité implicite (Dichotomie) : {:.4f}".format(implied_vol_bisection))

Volatilité implicite (Newton-Raphson) : 0.1880
Volatilité implicite (Dichotomie) : 0.1880


In [56]:
import copy

def nelder_mead(f, x_start,
                step=0.1, no_improve_thr=10e-6,
                no_improv_break=50, max_iter=0,
                alpha=1., gamma=2., rho=-0.5, sigma=0.5):
    '''
        @param f (function): function to optimize, must return a scalar score
            and operate over a numpy array of the same dimensions as x_start
        @param x_start (numpy array): initial position
        @param step (float): look-around radius in initial step
        @no_improv_thr,  no_improv_break (float, int): break after no_improv_break iterations with
            an improvement lower than no_improv_thr
        @max_iter (int): always break after this number of iterations.
            Set it to 0 to loop indefinitely.
        @alpha, gamma, rho, sigma (floats): parameters of the algorithm
            (see Wikipedia page for reference)

        return: tuple (best parameter array, best score)
    '''

    # init
    dim = len(x_start)
    prev_best = f(x_start)
    no_improv = 0
    res = [[x_start, prev_best]]

    for i in range(dim):
        x = copy.copy(x_start)
        x[i] = x[i] + step
        score = f(x)
        res.append([x, score])

    # simplex iter
    iters = 0
    while 1:
        # order
        res.sort(key=lambda x: x[1])
        best = res[0][1]

        # break after max_iter
        if max_iter and iters >= max_iter:
            return res[0]
        iters += 1

        # break after no_improv_break iterations with no improvement
        print (best)

        if best < prev_best - no_improve_thr:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break:
            return res[0]

        # centroid
        x0 = [0.] * dim
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)

        # reflection
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f(xr)
        if res[0][1] <= rscore < res[-2][1]:
            del res[-1]
            res.append([xr, rscore])
            continue

        # expansion
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f(xe)
            if escore < rscore:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue

        # contraction
        xc = x0 + rho*(x0 - res[-1][0])
        cscore = f(xc)
        if cscore < res[-1][1]:
            del res[-1]
            res.append([xc, cscore])
            continue

        # reduction
        x1 = res[0][0]
        nres = []
        for tup in res:
            redx = x1 + sigma*(tup[0] - x1)
            score = f(redx)
            nres.append([redx, score])
        res = nres


In [58]:
def f(x):
    n = 5
    sum = 0
    for i in range(0,n-1):
        sum += (1-x[i])**2 + 100*(x[i+1] - x[i]**2)**2

    return sum
print (nelder_mead(f, np.array([1., 5., 3., 7., 3.])))

261137.0
239533.729888
239533.729888
239533.729888
239533.729888
219659.2572377328
199006.75053924375
199006.75053924375
178861.46871569974
178861.46871569974
133164.29215731577
133164.29215731577
102835.52046519643
102835.52046519643
72575.34180162253
59201.796486540894
52676.930622373184
52676.930622373184
52676.930622373184
52676.930622373184
52676.930622373184
52676.930622373184
52676.930622373184
51609.835653398324
43989.49574588875
43989.49574588875
43989.49574588875
43989.49574588875
43989.49574588875
35568.11145749098
35568.11145749098
35568.11145749098
35568.11145749098
25477.74752117975
18643.474557548532
18643.474557548532
18643.474557548532
15179.55261320999
6118.778215693437
6118.778215693437
6118.778215693437
6118.778215693437
6118.778215693437
3382.5903113887603
3382.5903113887603
3382.5903113887603
3382.5903113887603
3279.4611418760205
3279.4611418760205
1425.830826670536
1425.830826670536
1425.830826670536
1425.830826670536
1425.830826670536
1425.830826670536
1425.8308

In [70]:
def f(x):
    return math.cos(math.pi*x[0])*math.sin(math.pi*x[1])*math.exp(-(x[0]**2+x[1]**2)/10)

print (nelder_mead(f, np.array([1., 1.])))

-1.0026559961204525e-16
-0.43039704141328394
-0.6913664461520033
-0.7243502972152517
-0.8279356329010793
-0.8279356329010793
-0.8777334620331074
-0.8777334620331074
-0.8777334620331074
-0.8777334620331074
-0.882986295925494
-0.882986295925494
-0.882986295925494
-0.8841768061338209
-0.8845308298320435
-0.8845308298320435
-0.8845642207401316
-0.8845642207401316
-0.8846469930221232
-0.8846622502612747
-0.8846792250804759
-0.8846792250804759
-0.8846813359456521
-0.8846868198005172
-0.8846875576154041
-0.8846877918507062
-0.8846888950469259
-0.8846890176837902
-0.8846890176837902
-0.8846893480466684
-0.8846893480466684
-0.8846893480466684
-0.8846893653333918
-0.8846894247660302
-0.8846894247660302
-0.8846894247660302
-0.8846894247660302
-0.8846894250255367
-0.8846894299197289
-0.8846894299197289
-0.8846894299197289
-0.8846894305721008
-0.8846894306186698
-0.8846894308131643
-0.8846894308131643
-0.8846894308131643
-0.8846894309025334
-0.8846894309025334
-0.8846894309025334
-0.884689430902533

Fonction de Styblinski Tang

In [71]:
def f(x):
    return (x[0]**4 + x[1]**4 - 16*(x[0]**2 + x[1]**2) + 5*(x[0]+x[1]))/2

print (nelder_mead(f, np.array([2.5, 3.])))

-48.73120000000001
-49.884446875
-50.016496875
-50.016496875
-50.02724687499999
-50.02724687499999
-50.04836600341797
-50.05353562922478
-50.055406525268964
-50.055406525268964
-50.05876724679747
-50.05876724679747
-50.05876724679747
-50.05876724679747
-50.05876724679747
-50.05876724679747
-50.05886407377825
-50.05886407377825
-50.05886407377825
-50.058875338744585
-50.058875338744585
-50.058885665302405
-50.058885665302405
-50.0588877287895
-50.05889144433604
-50.058891585029016
-50.05889249724085
-50.05889286779772
-50.05889301836936
-50.058893136582014
-50.058893275390815
-50.058893275390815
-50.058893275390815
-50.05889330495957
-50.05889330495957
-50.05889330495957
-50.05889330685548
-50.058893309730195
-50.058893309730195
-50.058893309730195
-50.05889331034196
-50.05889331049789
-50.05889331049789
-50.05889331050564
-50.05889331054861
-50.05889331054861
-50.0588933105632
-50.058893310564486
-50.058893310564486
-50.05889331056746
-50.05889331056746
-50.05889331056746
-50.058893310

Fonction d'Ackley

In [68]:
def f(x):
    n = 4
    return 20 + math.exp(1) - 20*(-0.2*np.sqrt((x[0]**2 + x[1]**2)/n)) - math.exp((math.cos(2*math.pi*x[0]) + math.cos(2*math.pi*x[1]))/n)

print (nelder_mead(f, np.array([-0.5, 0.1])))

22.542902953582576
22.38942680622937
21.900769188185873
21.900769188185873
21.50011189002199
21.50011189002199
21.165588777129884
21.165588777129884
21.165588777129884
21.165588777129884
21.165588777129884
21.165588777129884
21.096293260352848
21.096293260352848
21.096293260352848
21.096293260352848
21.08320414181524
21.08320414181524
21.080906369287987
21.080617702483256
21.074741737225853
21.072102293319183
21.072102293319183
21.072102293319183
21.07204046597651
21.071101128425223
21.070029623399964
21.070029623399964
21.069943238659455
21.069943238659455
21.069762956508036
21.069727391130712
21.06971798085416
21.069613338240288
21.069613338240288
21.069613338240288
21.06959903805765
21.069586706163648
21.069578320751447
21.069578320751447
21.0695677574175
21.0695677574175
21.0695677574175
21.069565546434234
21.06956323195965
21.069563187112312
21.069563187112312
21.06956187713273
21.06956115748173
21.06956115748173
21.06956115748173
21.06956115748173
21.06956115748173
21.06956093364